Connected to venv (Python 3.13.2)

In [1]:
import os
import django
from django.conf import settings
from django.apps import AppConfig

os.chdir('/Users/sg44574/Dropbox/Coding/nba-analytics-dashboard2')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'nba_analytics_project.settings')
django.setup()

In [2]:
"""
Django settings for nba_analytics_project project.

Generated by 'django-admin startproject' using Django 5.1.7.

For more information on this file, see
https://docs.djangoproject.com/en/5.1/topics/settings/

For the full list of settings and their values, see
https://docs.djangoproject.com/en/5.1/ref/settings/
"""

from pathlib import Path


# Quick-start development settings - unsuitable for production
# See https://docs.djangoproject.com/en/5.1/howto/deployment/checklist/


import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# SECURITY WARNING: keep the secret key used in production secret!
SECRET_KEY = os.getenv('SECRET_KEY')
# SECURITY WARNING: don't run with debug turned on in production!
DEBUG = os.getenv('DEBUG')

# Database configuration
DATABASES = {
    'default': {
        'ENGINE': 'django.db.backends.postgresql',
        'NAME': os.getenv('DB_NAME'),
        'USER': os.getenv('DB_USER'),
        'PASSWORD': os.getenv('DB_PASSWORD'),
        'HOST': os.getenv('DB_HOST'),
        'PORT': os.getenv('DB_PORT'),
    }
}



ALLOWED_HOSTS = []


# Application definition

INSTALLED_APPS = [
    # Default apps
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    
    # Third-party apps
    
    # Local apps (we'll create these next)
    'nba_data',
    'dashboard',
]

MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]

ROOT_URLCONF = "nba_analytics_project.urls"

TEMPLATES = [
    {
        "BACKEND": "django.template.backends.django.DjangoTemplates",
        "DIRS": [],
        "APP_DIRS": True,
        "OPTIONS": {
            "context_processors": [
                "django.template.context_processors.debug",
                "django.template.context_processors.request",
                "django.contrib.auth.context_processors.auth",
                "django.contrib.messages.context_processors.messages",
            ],
        },
    },
]

WSGI_APPLICATION = "nba_analytics_project.wsgi.application"


# Database
# https://docs.djangoproject.com/en/5.1/ref/settings/#databases

DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.sqlite3",
    }
}


# Password validation
# https://docs.djangoproject.com/en/5.1/ref/settings/#auth-password-validators

AUTH_PASSWORD_VALIDATORS = [
    {
        "NAME": "django.contrib.auth.password_validation.UserAttributeSimilarityValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.MinimumLengthValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.CommonPasswordValidator",
    },
    {
        "NAME": "django.contrib.auth.password_validation.NumericPasswordValidator",
    },
]


# Internationalization
# https://docs.djangoproject.com/en/5.1/topics/i18n/

LANGUAGE_CODE = "en-us"

TIME_ZONE = "UTC"

USE_I18N = True

USE_TZ = True


# Static files (CSS, JavaScript, Images)
# https://docs.djangoproject.com/en/5.1/howto/static-files/

STATIC_URL = "static/"

# Default primary key field type
# https://docs.djangoproject.com/en/5.1/ref/settings/#default-auto-field

DEFAULT_AUTO_FIELD = "django.db.models.BigAutoField"


In [3]:
import os
from typing import Dict, Any, List, Optional, Tuple
from balldontlie import BalldontlieAPI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class NBAApiClient:
    """Client for interacting with the balldontlie API using the official package"""
    
    def __init__(self):
        # Get API key from environment variable
        api_key = os.getenv('BALLDONTLIE_API_KEY')
        if not api_key:
            raise ValueError("BALLDONTLIE_API_KEY environment variable is required")
        
        self.api = BalldontlieAPI(api_key=api_key)
    
    def get_all_teams(self) -> List[Dict]:
        """Get all NBA teams"""
        response = self.api.nba.teams.list()
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBATeam objects to dictionaries
            teams = []
            for team in response.data:
                team_dict = {
                    'id': team.id,
                    'abbreviation': team.abbreviation,
                    'city': team.city,
                    'conference': team.conference,
                    'division': team.division,
                    'full_name': team.full_name,
                    'name': team.name
                }
                teams.append(team_dict)
            return teams
        return []
    
    def get_players(self, per_page: int = 25, cursor: Optional[int] = None) -> Tuple[List[Dict], Optional[int]]:
        """
        Get paginated list of players
        
        Args:
            per_page: Number of players per page
            cursor: Cursor for pagination (new pagination style)
            
        Returns:
            Tuple containing (list of player dictionaries, next cursor)
        """
        params = {
            'per_page': per_page,
            'cursor': cursor
        }
            
        response = self.api.nba.players.list(**params)
        
        # Default return values
        players = []
        next_cursor = None
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBAPlayer objects to dictionaries
            for player in response.data:
                # Extract team data if available
                team_data = None
                if hasattr(player, 'team') and player.team:
                    team_data = {
                        'id': player.team.id,
                        'abbreviation': player.team.abbreviation,
                        'city': player.team.city,
                        'conference': player.team.conference,
                        'division': player.team.division,
                        'full_name': player.team.full_name,
                        'name': player.team.name
                    }
                
                player_dict = {
                    'id': player.id,
                    'first_name': player.first_name,
                    'last_name': player.last_name,
                    'position': getattr(player, 'position', ''),
                    'height': getattr(player, 'height', None),
                    'weight_pounds': getattr(player, 'weight_pounds', None),
                    'jersey_number': getattr(player, 'jersey_number', None),
                    'college': getattr(player, 'college', None),
                    'country': getattr(player, 'country', None),
                    'draft_year': getattr(player, 'draft_year', None),
                    'draft_round': getattr(player, 'draft_round', None),
                    'draft_number': getattr(player, 'draft_number', None),
                    'team': team_data
                }
                players.append(player_dict)
        
        # Extract next cursor if available
        if hasattr(response, 'meta') and hasattr(response.meta, 'next_cursor'):
            next_cursor = response.meta.next_cursor
            
        return players, next_cursor
    
    def get_games(self, **params) -> Tuple[List[Dict], Optional[int]]:
        """
        Get games with optional filters
        
        Args:
            **params: Parameters to filter games by (per_page, cursor, seasons, team_ids, etc.)
            
        Returns:
            Tuple containing (list of game dictionaries, next cursor)
        """
        response = self.api.nba.games.list(**params)
        
        # Default return values
        games = []
        next_cursor = None
        
        # Handle ListResponse object - get the data attribute directly
        if hasattr(response, 'data'):
            # Convert NBAGame objects to dictionaries
            for game in response.data:
                # Extract home team data
                home_team = None
                if hasattr(game, 'home_team') and game.home_team:
                    home_team = {
                        'id': game.home_team.id,
                        'abbreviation': game.home_team.abbreviation,
                        'city': game.home_team.city,
                        'conference': game.home_team.conference,
                        'division': game.home_team.division,
                        'full_name': game.home_team.full_name,
                        'name': game.home_team.name
                    }
                
                # Extract visitor team data
                visitor_team = None
                if hasattr(game, 'visitor_team') and game.visitor_team:
                    visitor_team = {
                        'id': game.visitor_team.id,
                        'abbreviation': game.visitor_team.abbreviation,
                        'city': game.visitor_team.city,
                        'conference': game.visitor_team.conference,
                        'division': game.visitor_team.division,
                        'full_name': game.visitor_team.full_name,
                        'name': game.visitor_team.name
                    }
                
                game_dict = {
                    'id': game.id,
                    'date': game.date,
                    'datetime': getattr(game, 'datetime', None),
                    'home_team': home_team,
                    'visitor_team': visitor_team,
                    'home_team_score': game.home_team_score,
                    'visitor_team_score': game.visitor_team_score,
                    'season': getattr(game, 'season', None),
                    'status': game.status,
                    'period': getattr(game, 'period', None),
                    'time': getattr(game, 'time', None),
                    'postseason': getattr(game, 'postseason', False)
                }
                
                games.append(game_dict)
        
        # Extract next cursor if available
        if hasattr(response, 'meta') and hasattr(response.meta, 'next_cursor'):
            next_cursor = response.meta.next_cursor
            
        return games, next_cursor
    
    def _process_response(self, response) -> Dict:
        """Process API response into a dictionary format"""
        result = {}
        
        # Extract data from response
        if hasattr(response, 'data'):
            result['data'] = response.data
        
        # Extract meta information if available
        if hasattr(response, 'meta'):
            result['meta'] = {}
            meta = response.meta
            
            # Copy common meta fields
            for field in ['total_pages', 'current_page', 'next_page', 'per_page', 'total_count', 'next_cursor']:
                if hasattr(meta, field):
                    result['meta'][field] = getattr(meta, field)
        
        return result

# Create a singleton instance
api_client = NBAApiClient()

In [7]:
from django.db import models

class Team(models.Model):
    class Meta:
        app_label = 'nba_data'
    team_id = models.IntegerField(unique=True)  # API ID
    abbreviation = models.CharField(max_length=3)
    city = models.CharField(max_length=50)
    conference = models.CharField(max_length=10)
    division = models.CharField(max_length=20)
    full_name = models.CharField(max_length=100)
    name = models.CharField(max_length=50)
    
    def __str__(self):
        return self.full_name

class Player(models.Model):
    class Meta:
        app_label = 'nba_data'
    player_id = models.IntegerField(unique=True)  # API ID
    first_name = models.CharField(max_length=50)
    last_name = models.CharField(max_length=50)
    position = models.CharField(max_length=10, null=True, blank=True)
    height_feet = models.IntegerField(null=True, blank=True)
    height_inches = models.IntegerField(null=True, blank=True)
    height_total_inches = models.IntegerField(null=True, blank=True)  # For easy sorting
    weight_pounds = models.IntegerField(null=True, blank=True)
    jersey_number = models.CharField(max_length=10, null=True, blank=True)
    college = models.CharField(max_length=100, null=True, blank=True)
    country = models.CharField(max_length=100, null=True, blank=True)
    draft_year = models.IntegerField(null=True, blank=True)
    draft_round = models.IntegerField(null=True, blank=True)
    draft_number = models.IntegerField(null=True, blank=True)
    team = models.ForeignKey(Team, on_delete=models.SET_NULL, null=True, related_name='players')
    
    def __str__(self):
        return f"{self.first_name} {self.last_name}"
    
    @property
    def height_display(self) -> str:
        """Returns height in format '6-2'"""
        if self.height_feet is not None and self.height_inches is not None:
            return f"{self.height_feet}-{self.height_inches}"
        return ""

class Game(models.Model):
    class Meta:
        app_label = 'nba_data'
    game_id = models.IntegerField(unique=True)  # API ID
    date = models.DateField()
    datetime = models.DateTimeField(null=True, blank=True)
    home_team = models.ForeignKey(Team, on_delete=models.CASCADE, related_name='home_games')
    visitor_team = models.ForeignKey(Team, on_delete=models.CASCADE, related_name='away_games')
    home_team_score = models.IntegerField()
    visitor_team_score = models.IntegerField()
    season = models.IntegerField()
    status = models.CharField(max_length=20)
    period = models.IntegerField(null=True, blank=True)
    time = models.CharField(max_length=10, null=True, blank=True)
    postseason = models.BooleanField(default=False)
    
    def __str__(self):
        return f"{self.visitor_team} @ {self.home_team} ({self.date})"

/Users/sg44574/Dropbox/Coding/nba-analytics-dashboard2/venv/lib/python3.13/site-packages/django/db/models/base.py:368: RuntimeWarning: Model 'nba_data.player' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/Users/sg44574/Dropbox/Coding/nba-analytics-dashboard2/venv/lib/python3.13/site-packages/django/db/models/base.py:368: RuntimeWarning: Model 'nba_data.game' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


In [14]:
pip install balldontlie


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Verify the API key is loaded
api_key = os.getenv('BALLDONTLIE_API_KEY')
print(f"API key loaded: {'Yes' if api_key else 'No'}")

API key loaded: Yes


In [20]:
from django.core.management.base import BaseCommand
import logging

logger = logging.getLogger(__name__)

class Command(BaseCommand):
    help = 'Import all NBA teams from the API'

    def handle(self, *args, **options):
        self.stdout.write('Importing NBA teams...')
        
        # Get teams from API
        try:
            teams_data = NBAApiClient().get_all_teams()
            
            if not teams_data:
                self.stdout.write(self.style.WARNING('No teams found in the API response.'))
                return
            
            self.stdout.write(f'Found {len(teams_data)} teams')
            
            # Process each team
            for team_data in teams_data:
                team, created = Team.objects.update_or_create(
                    team_id=team_data['id'],
                    defaults={
                        'abbreviation': team_data['abbreviation'],
                        'city': team_data['city'],
                        'conference': team_data['conference'],
                        'division': team_data['division'],
                        'full_name': team_data['full_name'],
                        'name': team_data['name'],
                    }
                )
                
                action = 'Created' if created else 'Updated'
                self.stdout.write(f'{action} team: {team.full_name}')
            
            self.stdout.write(self.style.SUCCESS('Successfully imported teams'))
            
        except Exception as e:
            self.stdout.write(self.style.ERROR(f'Error importing teams: {str(e)}'))
            logger.error(f'Error importing teams: {str(e)}', exc_info=True)

In [23]:
# Create the API client instance
api_client = NBAApiClient()

# Get teams data and store it in variable
teams_data = api_client.get_all_teams()

# Debug print
print(f"Number of teams found: {len(teams_data)}")
print("\nFirst team data:", teams_data[0] if teams_data else "No teams found")

Number of teams found: 45

First team data: {'id': 1, 'abbreviation': 'ATL', 'city': 'Atlanta', 'conference': 'East', 'division': 'Southeast', 'full_name': 'Atlanta Hawks', 'name': 'Hawks'}


In [27]:
teams_data

[{'id': 1,
  'abbreviation': 'ATL',
  'city': 'Atlanta',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Atlanta Hawks',
  'name': 'Hawks'},
 {'id': 2,
  'abbreviation': 'BOS',
  'city': 'Boston',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Boston Celtics',
  'name': 'Celtics'},
 {'id': 3,
  'abbreviation': 'BKN',
  'city': 'Brooklyn',
  'conference': 'East',
  'division': 'Atlantic',
  'full_name': 'Brooklyn Nets',
  'name': 'Nets'},
 {'id': 4,
  'abbreviation': 'CHA',
  'city': 'Charlotte',
  'conference': 'East',
  'division': 'Southeast',
  'full_name': 'Charlotte Hornets',
  'name': 'Hornets'},
 {'id': 5,
  'abbreviation': 'CHI',
  'city': 'Chicago',
  'conference': 'East',
  'division': 'Central',
  'full_name': 'Chicago Bulls',
  'name': 'Bulls'},
 {'id': 6,
  'abbreviation': 'CLE',
  'city': 'Cleveland',
  'conference': 'East',
  'division': 'Central',
  'full_name': 'Cleveland Cavaliers',
  'name': 'Cavaliers'},
 {'id': 7,
  'abbreviati